In [2]:
%%configure -f
{ "conf":{    
          "spark.databricks.hive.metastore.glueCatalog.enabled" : "true",
          "spark.jars.packages": "io.delta:delta-core_2.12:0.8.0",
          "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
          "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog",
          "spark.sql.warehouse.dir": "s3://aws-poc-serverless-analytics/dev/clean/delta_dw/"
    
         }
}


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1619432290004_0014,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1619432290004_0014,pyspark,idle,Link,Link,✔


In [3]:
from pyspark.sql.functions import *
from delta.tables import *
spark.sql("SET spark.databricks.delta.schema.autoMerge.enabled = true") 
#import boto3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[key: string, value: string]

In [6]:
target_table = "merchants_cln"
stg_table = "merchants_cln_stg"
database = "delta_lake_demo"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
spark.sql(f"DROP TABLE IF EXISTS {target_table}")
#spark.sql(f"DROP TABLE IF EXISTS {stg_table}")
spark.sql(f"DROP TABLE IF EXISTS merchants_cln_external")
spark.sql(f"DROP DATABASE IF EXISTS {database}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [8]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {database}")
spark.sql(f"USE {database}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [27]:
col_schema_list = spark.sql("""select * from merchants_cln""").dtypes
col_schema_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('id', 'string'), ('company_name', 'string'), ('telephone', 'string'), ('state', 'string'), ('created_at', 'string'), ('updated_at', 'string'), ('active', 'string')]

In [28]:
partition_key_list = ['state']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
## taking partition_key schema out from table schema 
partition_key_schema = ''
for i in partition_key_list:
    for schema in col_schema_list:
        if schema[0] == i:
            #partition_key_schema.append(i + ' ' + schema[1])
            partition_key_schema+=i + ' ' + schema[1] + ','
            col_schema_list.remove(schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
partition_key_schema[:-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'state string'

In [31]:
col_schema_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('id', 'string'), ('company_name', 'string'), ('telephone', 'string'), ('created_at', 'string'), ('updated_at', 'string'), ('active', 'string')]

In [32]:
## creating schema in python string format
col_type_schema = ''
for schema in col_schema_list:
    col_type_schema += schema[0] + ' ' + schema[1] + ','

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
col_type_schema[:-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'id string,company_name string,telephone string,created_at string,updated_at string,active string'

In [19]:
## reading csv file from csv
inputPath = "s3://aws-poc-serverless-analytics/dev/flattened_raw/mr/"
input_df = spark.read.csv(inputPath, header=True, sep=',')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
## overwriting as staging table
source_staging_df = input_df.withColumn("active", lit("true"))
source_staging_df.write.format("delta").mode('overwrite').option("overwriteSchema", "true").saveAsTable(target_table)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
## this is done to create sylink file , which is used by external tables to integrate with athena
spark.sql(f"""GENERATE symlink_format_manifest FOR TABLE {target_table}""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [40]:
## creating external table with the help of symlink file so that table written to 
## target location could be read in athena
spark.sql(f""" CREATE EXTERNAL TABLE IF NOT EXISTS {target_table}_external {col_type_schema[:-1]})
PARTITIONED BY ({partition_key_schema[:-1]})
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'  ---symlink file format
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'  --output file format, readable in athena
LOCATION 's3://aws-poc-serverless-analytics/dev/clean/delta_dw/delta_lake_demo.db/merchants_cln/_symlink_format_manifest/' 
""")  ## location of symlink file

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:

mismatched input 'id' expecting {<EOF>, ';'}(line 1, pos 60)

== SQL ==
 CREATE EXTERNAL TABLE IF NOT EXISTS merchants_cln_external id string,company_name string,telephone string,created_at string,updated_at string,active string)
------------------------------------------------------------^^^
PARTITIONED BY (state string)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://aws-poc-serverless-analytics/dev/clean/delta_dw/delta_lake_demo.db/merchants_cln/_symlink_format_manifest/' 

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 649, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1305, in __call__
 